In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import datetime
from PIL import Image, ImageStat
import math
from glob import glob
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Add
from tensorflow.keras.initializers import RandomNormal
from tifffile import tifffile
# %load_ext tensorboard

In [2]:
# Enable GPU and check available devices
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


In [4]:
from tqdm.notebook import tqdm

In [5]:
kidney5_images = glob('/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/*.tif')
kidney6_images = glob('/kaggle/input/blood-vessel-segmentation/test/kidney_6/images/*.tif')

In [6]:
kidney5_df=pd.DataFrame([kidney5_images],index=['images']).T
kidney6_df=pd.DataFrame([kidney6_images],index=['images']).T

In [7]:
test=pd.concat([kidney5_df,kidney6_df],axis=0)
test=test.sort_values('images').reset_index(drop=True)

In [8]:
def encode_image_to_tiles(image, tile_size):

    # Calculate the number of tiles along rows and columns
    rows, cols = image.shape
    m, n = tile_size
    num_rows = rows // m
    num_cols = cols // n

    # Create a list to store tiles
    tiles = []

    # Iterate over the image and extract tiles
    for i in range(num_rows):
        for j in range(num_cols):
            tile = image[i * m: (i + 1) * m, j * n: (j + 1) * n]
            tiles.append(tile)

    return tiles

def decode_tiles_to_image(tiles, original_shape):
    # Calculate the number of tiles along rows and columns
    num_rows, num_cols = original_shape
    m, n = tiles[0].shape

    # Create an empty image to reconstruct
    reconstructed_image = np.zeros(original_shape, dtype=np.float16)

    # Iterate over the tiles and place them in the reconstructed image
    idx = 0
    for i in range(num_rows//m):
        for j in range(num_cols//n):
            tile = tiles[idx]
            reconstructed_image[i * m: (i + 1) * m, j * n: (j + 1) * n] = tile
            idx += 1

    return reconstructed_image

In [9]:
def rle_encode(mask):
    pixel = mask.flatten()
    pixel = np.concatenate([[0], pixel, [0]])
    run = np.where(pixel[1:] != pixel[:-1])[0] + 1
    run[1::2] -= run[::2]
    rle = ' '.join(str(r) for r in run)
    if rle == '':
        rle = '1 0'
    return rle

In [10]:
# Hyperparameters
BN_MOMENTUM = 0.1
BN_EPSILON = 1e-5
INITIALIZER = 'he_normal'

In [11]:
# Functions to build layers
def conv(x, outsize, kernel_size, strides_=1, padding_='same', activation=None):
    return Conv2D(outsize, kernel_size, strides=strides_, padding=padding_, 
                  kernel_initializer=INITIALIZER, use_bias=False, 
                  activation=activation)(x)

def BasicBlock(x, size, downsampe=False):
    residual = x

    out = conv(x, size, 3)
    out = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(out)
    out = Activation('relu')(out)

    out = conv(out, size, 3)
    out = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(out)

    if downsampe:
        residual = conv(x, size, 1, padding_='valid')
        residual = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(residual)

    out = Add()([out, residual])
    out = Activation('relu')(out)

    return out

def BottleNeckBlock(x, size, downsampe=False):
    residual = x

    out = conv(x, size, 1, padding_='valid')
    out = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(out)
    out = Activation('relu')(out)

    out = conv(out, size, 3)
    out = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(out)
    out = Activation('relu')(out)

    out = conv(out, size * 4, 1, padding_='valid')
    out = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(out)

    if downsampe:
        residual = conv(x, size * 4, 1, padding_='valid')
        residual = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(residual)

    out = Add()([out, residual])
    out = Activation('relu')(out)

    return out

def layer1(x):
    x = BottleNeckBlock(x, 64, downsampe=True)
    x = BottleNeckBlock(x, 64)
    x = BottleNeckBlock(x, 64)
    x = BottleNeckBlock(x, 64)

    return x

def transition_layer(x, in_channels, out_channels):
    num_in = len(in_channels)
    num_out = len(out_channels)
    out = []

    for i in range(num_out):
        if i < num_in:
            if in_channels[i] != out_channels[i]:
                residual = conv(x[i], out_channels[i], 3)
                residual = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(residual)
                residual = Activation('relu')(residual)
                out.append(residual)
            else:
                out.append(x[i])
        else:
            residual = conv(x[-1], out_channels[i], 3, strides_=2)
            residual = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(residual)
            residual = Activation('relu')(residual)
            out.append(residual)

    return out

def branches(x, block_num, channels):
    out = []
    for i in range(len(channels)):
        residual = x[i]
        for j in range(block_num):
            residual = BasicBlock(residual, channels[i])
        out.append(residual)
    return out

def fuse_layers(x, channels, multi_scale_output=True):
    out = []

    for i in range(len(channels) if multi_scale_output else 1):
        residual = x[i]
        for j in range(len(channels)):
            if j > i:
                y = conv(x[j], channels[i], 1, padding_='valid')
                y = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(y)
                y = UpSampling2D(size=2 ** (j - i))(y)
                residual = Add()([residual, y])
            elif j < i:
                y = x[j]
                for k in range(i - j):
                    if k == i - j - 1:
                        y = conv(y, channels[i], 3, strides_=2)
                        y = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(y)
                    else:
                        y = conv(y, channels[j], 3, strides_=2)
                        y = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(y)
                        y = Activation('relu')(y)
                residual = Add()([residual, y])

        residual = Activation('relu')(residual)
        out.append(residual)

    return out

In [12]:
# Functions to create model
def HighResolutionModule(x, channels, multi_scale_output=True):
    residual = branches(x, 4, channels)
    out = fuse_layers(residual, channels, multi_scale_output=multi_scale_output)
    return out


def stage(x, num_modules, channels, multi_scale_output=True):
    out = x
    for i in range(num_modules):
        if i == num_modules - 1 and multi_scale_output == False:
            out = HighResolutionModule(out, channels, multi_scale_output=False)
        else:
            out = HighResolutionModule(out, channels)

    return out


def hrnet_keras(input_size=(512,512, 1)):
    channels_2 = [32, 64]
    channels_3 = [32, 64, 128]
    channels_4 = [32, 64, 128, 256]
    num_modules_2 = 1
    num_modules_3 = 4
    num_modules_4 = 3

    inputs = Input(input_size)
    x = conv(inputs, 64, 3, strides_=2)
    x = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(x)
    x = conv(x, 64, 3, strides_=2)
    x = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(x)
    x = Activation('relu')(x)

    la1 = layer1(x)
    tr1 = transition_layer([la1], [256], channels_2)
    st2 = stage(tr1, num_modules_2, channels_2)
    tr2 = transition_layer(st2, channels_2, channels_3)
    st3 = stage(tr2, num_modules_3, channels_3)
    tr3 = transition_layer(st3, channels_3, channels_4)
    st4 = stage(tr3, num_modules_4, channels_4, multi_scale_output=False)
    up1 = UpSampling2D()(st4[0])
    up1 = conv(up1, 32, 3)
    up1 = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(up1)
    up1 = Activation('relu')(up1)
    up2 = UpSampling2D()(up1)
    up2 = conv(up2, 32, 3)
    up2 = BatchNormalization(epsilon=BN_EPSILON, momentum=BN_MOMENTUM)(up2)
    up2 = Activation('relu')(up2)
    final = conv(up2, 1, 1, padding_='valid', activation='sigmoid')

    model = Model(inputs=inputs, outputs=final)

    return model


In [13]:
with strategy.scope():
    model = hrnet_keras()

# Load weghts

In [14]:
model.load_weights('/kaggle/input/kidney-train-1/model_weights_0.9115426540374756_0.14422346651554108.h5')

In [15]:
result=[]
for img_path in tqdm(test.images):
    img=tifffile.imread(img_path)
    img=img/2**16
#     print(img.min(),img.max())
#     Min-max normalization
    min_value = np.min(img)
    max_value = np.max(img)

    img = (img - min_value) / (max_value - min_value+1e-9)
    height,width=img.shape
    if height/512==height//512:
        rows=height/512
    else:
        rows=height//512+1
    if width/512==width//512:
        columns=width/512
    else:
        columns=width//512+1
    img=np.pad(img,((0,rows*512-height),(0,columns*512-width)),mode='constant',constant_values=0.2)
    img_list=encode_image_to_tiles(img,(512,512))
    tf_img=tf.convert_to_tensor(img_list)
    pred=model.predict(tf_img)
    pred=np.squeeze(pred)
    pred=np.where(pred>0.4,1,0)
    pred=decode_tiles_to_image(pred,(rows*512,columns*512))
    pred=pred[:height,:width]
    encode=rle_encode(pred)
    result.append(encode)

  0%|          | 0/6 [00:00<?, ?it/s]

1/1 [==============================] - 0s 235ms/step


In [16]:
test['id']=test.images.apply(lambda x:x.split('/')[5]+'_'+x.split('/')[7][:-4])

In [17]:
test['rle']=result

In [18]:
test

,images,id,rle
0,/kaggle/input/blood-vessel-segmentation/test/k...,kidney_5_0000,1 0
1,/kaggle/input/blood-vessel-segmentation/test/k...,kidney_5_0001,1 0
2,/kaggle/input/blood-vessel-segmentation/test/k...,kidney_5_0002,1 0
3,/kaggle/input/blood-vessel-segmentation/test/k...,kidney_6_0000,1 0
4,/kaggle/input/blood-vessel-segmentation/test/k...,kidney_6_0001,1 0
5,/kaggle/input/blood-vessel-segmentation/test/k...,kidney_6_0002,1 0


In [19]:
test=test[['id','rle']]

In [20]:
test.to_csv('submission.csv',index=False)